In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer

train = pd.read_csv('salary-train.csv')
test = pd.read_csv('salary-test-mini.csv')

In [2]:
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [3]:
train.shape

(60000, 4)

In [4]:
#приводим все строки в нижний регистри для дальнейшего
#более корректного разделения текста на слова
train.FullDescription = train.FullDescription.str.lower()
train.LocationNormalized = train.LocationNormalized.str.lower()
train.ContractTime = train.ContractTime.str.lower()
#test.FullDescription = test.FullDescription.str.lower()
#test.LocationNormalized = test.LocationNormalized.str.lower()
#test.ContractTime = test.ContractTime.str.lower()

In [5]:
#по аналогичным соображениям оставим в тексте только символьно-буквенные выражения
train.FullDescription = train.FullDescription.replace('[^a-z0-9]', ' ', regex = True)
train.LocationNormalized = train.LocationNormalized.replace('[^a-z0-9]', ' ', regex = True)
train.ContractTime = train.ContractTime.replace('[^a-z0-9]', ' ', regex = True)

#test.FullDescription = test.FullDescription.replace('[^a-z0-9]', ' ', regex = True)
#test.LocationNormalized = test.LocationNormalized.replace('[^a-z0-9]', ' ', regex = True)
#test.ContractTime = test.ContractTime.replace('[^a-z0-9]', ' ', regex = True)

In [6]:
#преобразуем текста в векторы признаки
#учитывая факт,что оставим те слова, встречащиеся хотя бы в 5 объектах
vectorizer = TfidfVectorizer(min_df=5)
train_scaled = vectorizer.fit_transform(train.FullDescription)
test_scaled = vectorizer.transform(test.FullDescription)

In [7]:
#заменим пропуски в столбцах на nan
train.LocationNormalized.fillna('nan', inplace=True)
train.ContractTime.fillna('nan', inplace=True)

In [8]:
#воспользуемся one-hot кодирование соответствующих признаков
#опять же, чтоб перевести слова в числа
enc = DictVectorizer()
train_categ = enc.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))
test_categ = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [9]:
#объединим полученные переведенные текст и слова в числовые признаки
from scipy.sparse import hstack
edit_features_train = hstack([train_scaled, train_categ])

In [10]:
#обучим гребневую регрессию
clf = Ridge(alpha=1, random_state=241)
clf.fit(edit_features_train, train.SalaryNormalized)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [11]:
#предскажем значения на тестовой выборке и выведем их
edit_test = hstack([test_scaled, test_categ])
z = clf.predict(edit_test)
z_round =  [round(x,2) for x in z] 
print(z_round)
#отработала за неск секунд
#test WITHOUT str.lower and regex
#[55850.59, 37182.51]
#test WITH str.lower and regex
#такой ответ принят не был
#[56555.62, 37188.32]

[55850.59, 37182.51]


In [47]:
'''
from sklearn.linear_model import Lasso
clf = Lasso(alpha=1, random_state=241)
#clf.fit(edit_features_train, train.SalaryNormalized)
edit_test = hstack([test_scaled, test_categ])
z = clf.predict(edit_test)
z_round =  [round(x,2) for x in z] 
print(z_round)
#[55607.38, 36989.37]
эта часть кода работала минут 5
'''

'\nfrom sklearn.linear_model import Lasso\nclf = Lasso(alpha=1, random_state=241)\n#clf.fit(edit_features_train, train.SalaryNormalized)\nedit_test = hstack([test_scaled, test_categ])\nz = clf.predict(edit_test)\nz_round =  [round(x,2) for x in z] \nprint(z_round)\n#[55607.38, 36989.37]\nэта часть кода работала минут 5\n'